In [14]:
import squigglepy as sq
import numpy as np
import pandas as pd
from squigglepy.numbers import K, M, B
from chip_estimates_utils import (
    normalize_shares,
    compute_h100_equivalents,
    export_quarterly_by_version,
    print_cumulative_summary,
    estimate_chip_sales
)

sq.set_seed(42)
np.random.seed(42)
N_SAMPLES = 5000
H100_TOPS = 1979

In [ ]:
# ======================
# Specs and data inputs
# ======================

# TPU specs: 8-bit TOPS and manufacturing costs
TPU_SPECS = {
    'v3':  {'tops': 123,  'cost': sq.to(940, 1400)},
    'v4':  {'tops': 275,  'cost': sq.to(1100, 1500)},
    'v5e': {'tops': 393,  'cost': sq.to(950, 1400)},
    'v5p': {'tops': 918,  'cost': sq.to(2300, 2900)},
    'v6e': {'tops': 1836, 'cost': sq.to(1600, 1900)},
    'v7':  {'tops': 4614, 'cost': sq.to(4600, 5500)},
}

# Broadcom margins (higher in FY23, lower afterward)
MARGIN_FY23 = sq.to(0.60, 0.75)
MARGIN = sq.to(0.50, 0.70)

# ==========================
# Quarterly data/estimates on TPU revenue by quarter and production mix
# To do: figure out a more friendly way to store this data, e.g. CSV or Google sheet import
# ==========================

# TPU revenue to Broadcom by quarter ($ billions, p5-p95 ranges)
TPU_REVENUE = {
    'Q1_FY23': sq.to(0.42, 0.53), 'Q2_FY23': sq.to(0.53, 0.66),
    'Q3_FY23': sq.to(0.53, 0.66), 'Q4_FY23': sq.to(0.79, 0.99),
    'Q1_FY24': sq.to(1.25, 1.40), 'Q2_FY24': sq.to(1.65, 1.90),
    'Q3_FY24': sq.to(1.85, 2.00), 'Q4_FY24': sq.to(2.10, 2.30),
    'Q1_FY25': sq.to(2.20, 2.50), 'Q2_FY25': sq.to(2.40, 2.80),
    'Q3_FY25': sq.to(2.90, 3.20), 'Q4_FY25': sq.to(3.60, 4.00),
}

# Production mix by quarter (p5-p95 ranges, will be normalized later)
PROD_MIX = {
    'Q1_FY23': {'v4': sq.to(0.70, 0.85), 'v3': sq.to(0.15, 0.30)},
    'Q2_FY23': {'v4': sq.to(0.90, 0.99), 'v3': sq.to(0.01, 0.10)},
    'Q3_FY23': {'v4': sq.to(0.87, 0.98), 'v5e': sq.to(0.01, 0.08)},
    'Q4_FY23': {'v4': sq.to(0.80, 0.92), 'v5e': sq.to(0.08, 0.20)},
    'Q1_FY24': {'v5e': sq.to(0.35, 0.60), 'v4': sq.to(0.35, 0.60), 'v5p': sq.to(0.02, 0.08)},
    'Q2_FY24': {'v5e': sq.to(0.65, 0.85), 'v5p': sq.to(0.15, 0.30)},
    'Q3_FY24': {'v5e': sq.to(0.30, 0.70), 'v5p': sq.to(0.25, 0.60), 'v6e': sq.to(0.01, 0.03)},
    'Q4_FY24': {'v5e': sq.to(0.30, 0.55), 'v5p': sq.to(0.30, 0.55), 'v6e': sq.to(0.01, 0.08)},
    'Q1_FY25': {'v5e': sq.to(0.15, 0.40), 'v5p': sq.to(0.15, 0.40), 'v6e': sq.to(0.10, 0.25)},
    'Q2_FY25': {'v5e': sq.to(0.20, 0.35), 'v5p': sq.to(0.20, 0.35), 'v6e': sq.to(0.30, 0.45)},
    'Q3_FY25': {'v5e': sq.to(0.10, 0.30), 'v5p': sq.to(0.10, 0.35), 'v6e': sq.to(0.35, 0.65), 'v7': sq.to(0.01, 0.10)},
    'Q4_FY25': {'v5e': sq.to(0.01, 0.25), 'v5p': sq.to(0.05, 0.25), 'v6e': sq.to(0.40, 0.80), 'v7': sq.to(0.10, 0.20)},
}

In [ ]:
def get_price_distribution(version, is_fy23=False):
    """Get price distribution for a TPU version: cost / (1 - margin)."""
    margin = MARGIN_FY23 if is_fy23 else MARGIN
    return TPU_SPECS[version]['cost'] / (1 - margin)

# Pre-compute price distributions for each version and margin regime
PRICE_DIST_FY23 = {version: get_price_distribution(version, is_fy23=True) for version in TPU_SPECS}
PRICE_DIST = {version: get_price_distribution(version, is_fy23=False) for version in TPU_SPECS}

# Define sampling functions which get passed into estimate_chip_sales
def sample_revenue(quarter):
    # draw one sample from the quarter's distribution
    return (TPU_REVENUE[quarter] @ 1) * B

def sample_shares(quarter):
    mix = PROD_MIX[quarter]
    # draw one sample from each version's distribution
    raw_shares = {version: dist @ 1 for version, dist in mix.items()}
    return normalize_shares(raw_shares)

def sample_price(quarter, version):
    price_dists = PRICE_DIST_FY23 if 'FY23' in quarter else PRICE_DIST
    return price_dists[version] @ 1

In [ ]:
"""
Run Monte Carlo simulation to estimate chip volumes.

Args:
    quarters: list of quarter identifiers (e.g., ['Q1_FY23', 'Q2_FY23', ...])
    versions: list of chip types (e.g., ['v3', 'v4', 'v5e', ...])
    sample_revenue: fn(quarter) -> float, samples or looks up total chip revenue in dollars for a quarter
    sample_shares: fn(quarter) -> dict, samples {version: share} for a quarter (should sum to 1)
    sample_price: fn(quarter, version) -> float, samples or looks up price for a chip type in a quarter
    n_samples: number of Monte Carlo samples

Returns:
    Dictionary of {quarter: {version: [array of samples of chip unit counts]}}
    To find median, confidence intervals, etc you will need to take the percentiles of the result

Note on cross-quarter correlations:
    The sampling functions are called independently for each quarter within each iteration.
    This means any parameters you want correlated across quarters (e.g., a single margin
    value affecting all quarters) will NOT be correlated by default. To preserve cross-quarter
    correlations, pre-sample those parameters outside this function and have your sampling
    functions reference them.
"""

sim_results = estimate_chip_sales(
    quarters=list(TPU_REVENUE.keys()),
    versions=list(TPU_SPECS.keys()),
    sample_revenue=sample_revenue,
    sample_shares=sample_shares,
    sample_price=sample_price,
    n_samples=N_SAMPLES
)

In [20]:
# Summarize quarterly results
def summarize_results(results):
    """Create summary DataFrame with percentiles."""
    rows = []
    for quarter in results:
        row = {'Quarter': quarter}
        total = np.zeros(N_SAMPLES)
        for version in TPU_SPECS:
            arr = np.array(results[quarter][version])
            total += arr
            if arr.sum() > 0:
                row[f'{version}_p50'] = int(np.percentile(arr, 50))
        row['total_p5'] = int(np.percentile(total, 5))
        row['total_p50'] = int(np.percentile(total, 50))
        row['total_p95'] = int(np.percentile(total, 95))
        rows.append(row)
    return pd.DataFrame(rows)

df = summarize_results(sim_results)
print("TPU Production Volumes by Quarter (chips)")
print(df[['Quarter', 'total_p5', 'total_p50', 'total_p95']].to_string(index=False))

TPU Production Volumes by Quarter (chips)
Quarter  total_p5  total_p50  total_p95
Q1_FY23     94582     123987     156579
Q2_FY23    110291     152013     200039
Q3_FY23    111361     152199     200650
Q4_FY23    172804     230241     296284
Q1_FY24    341139     433589     537911
Q2_FY24    398180     541855     716102
Q3_FY24    379633     502411     650688
Q4_FY24    428072     557664     704946
Q1_FY25    461886     581091     726640
Q2_FY25    514790     636322     768427
Q3_FY25    571599     711759     857513
Q4_FY25    614335     786170     977324


In [ ]:
# Cumulative totals by TPU version
# 
# Note: don't trust the confidence intervals here, because they don't account for correlation across quarters
# This means they are probably too narrow

cumulative = {version: np.zeros(N_SAMPLES) for version in TPU_SPECS}
for quarter in sim_results:
    for version in TPU_SPECS:
        cumulative[version] += np.array(sim_results[quarter][version])

print_cumulative_summary(cumulative, TPU_SPECS, "Cumulative TPU Production (FY23-FY25)")


Cumulative TPU Production (FY23-FY25)
Version           p5          p50          p95
---------------------------------------------
v3           23,091       34,990       53,504
v4          671,743      784,420      907,928
v5e       2,011,863    2,384,657    2,809,608
v5p         680,345      814,893      975,846
v6e       1,092,037    1,340,499    1,602,458
v7           36,104       58,457       88,792
---------------------------------------------
TOTAL     5,043,151    5,434,319    5,856,804


In [22]:
# H100 equivalents (based on 8-bit TOPS)
h100_eq = compute_h100_equivalents(cumulative, TPU_SPECS, H100_TOPS)
print_cumulative_summary(h100_eq, TPU_SPECS, "H100 Equivalents (8-bit TOPS basis)")


H100 Equivalents (8-bit TOPS basis)
Version           p5          p50          p95
---------------------------------------------
v3            1,435        2,174        3,325
v4           93,344      109,002      126,164
v5e         399,526      473,557      557,946
v5p         315,592      378,005      452,666
v6e       1,013,128    1,243,636    1,486,667
v7           84,176      136,292      207,019
---------------------------------------------
TOTAL     2,132,162    2,350,774    2,578,384


In [23]:
# Fiscal year totals
def fiscal_year_totals(results):
    fy_totals = {'FY23': np.zeros(N_SAMPLES), 'FY24': np.zeros(N_SAMPLES), 'FY25': np.zeros(N_SAMPLES)}
    for quarter in results:
        fy = quarter.split('_')[1]
        for version in TPU_SPECS:
            fy_totals[fy] += np.array(results[quarter][version])
    return fy_totals

fy = fiscal_year_totals(sim_results)
print("\nTotal TPU Production by Fiscal Year")
for year in ['FY23', 'FY24', 'FY25']:
    p5, p50, p95 = [int(np.percentile(fy[year], p)) for p in [5, 50, 95]]
    print(f"{year}: {p50:,} chips (90% CI: {p5:,} - {p95:,})")


Total TPU Production by Fiscal Year
FY23: 661,984 chips (90% CI: 570,401 - 759,116)
FY24: 2,049,259 chips (90% CI: 1,787,363 - 2,318,646)
FY25: 2,724,755 chips (90% CI: 2,437,626 - 3,027,626)


In [ ]:
# Export quarterly volumes by version to CSV (with H100 equivalents)
# This isn't adapted for Airtable yet
export_df = export_quarterly_by_version(
    sim_results, TPU_SPECS, 'tpu_volumes_by_quarter_version.csv', N_SAMPLES, H100_TOPS
)
print(export_df.to_string(index=False))

quarter version  chips_p5  chips_p50  chips_p95  h100e_p5  h100e_p50  h100e_p95
Q1_FY23      v3     18318      28836      43469      1138       1792       2701
Q1_FY23      v4     66952      94207     125829      9303      13091      17485
Q2_FY23      v3      1665       5252      16457       103        326       1022
Q2_FY23      v4    103641     145252     193863     14401      20184      26939
Q3_FY23      v4    104614     146182     195254     14537      20313      27132
Q3_FY23     v5e      1655       4976      14050       328        988       2790
Q4_FY23      v4    140992     195700     262140     19592      27194      36426
Q4_FY23     v5e     18833      32111      52321      3740       6376      10390
Q1_FY24      v4    132150     197871     274895     18363      27496      38199
Q1_FY24     v5e    147875     222093     318378     29365      44104      63225
Q1_FY24     v5p      4028       8513      17742      1868       3949       8230
Q2_FY24     v5e    335068     478489    